# Raw Images to Grayscale

This notebook converts the raw images in S3 to grayscale versions and loads them to a different folder in the same bucket.

In [196]:
import cv2
import numpy as np
import boto3

s3_client = boto3.client('s3')
response = s3_client.list_objects(Bucket='bcbs-logo-training-images', Prefix='raw_images')    
contents=response['Contents']
def get_key(d):
    return d['Key']    
keys = list(map(get_key, contents))[1:]

print("Number of raw images: {}".format(len(gray_keys)))

Number of raw images: 389


In [ ]:
def convert_to_gray(s3_key):
    # defining s3 bucket object
    s3 = boto3.client("s3")
    bucket_name = "bcbs-logo-training-images"
    # fetching object from bucket
    file_obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
    # reading the file content in bytes
    file_content = file_obj["Body"].read()
    # creating 1D array from bytes data range between[0,255]
    np_array = np.fromstring(file_content, np.uint8)
    # decoding array
    image_np = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
    # converting image from RGB to Grayscale
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    return gray

gray_imgs = list(map(convert_to_gray, keys))

In [ ]:
plt.imshow(gray_imgs[0], 'gray')

In [193]:
def write_to_s3(img, key):
    from PIL import Image
    import io
    img = Image.fromarray(img).convert('RGB')
    out_img = io.BytesIO()
    img.save(out_img, format='png')
    out_img.seek(0)  
    return s3.Bucket('bcbs-logo-training-images').put_object(Key=key,Body=out_img,ContentType='image/png') 

def replace_keyname(key):
    return key.replace('raw_images', 'gray_images')

gray_keys = list(map(replace_keyname, keys))

print("Gray image keys: {}".format(len(gray_keys)))
print("Gray images: {}".format(len(gray_imgs)))

Gray image keys: 389
Gray images: 389


In [195]:
[write_to_s3(x, y) for x, y in zip(gray_imgs, gray_keys)]

[s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/002.jpg'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/090315_BCBSopening51.jpg'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/10-10_bcbs-kansas-solutions-inc-id-card-samples.png'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/100-1024x536.jpg'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/102726425-64427_10151606029368653_471083024_n.jpg'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/10899709-34f0-4965-b9bf-af0bcfe72ccb-Letter.jpg'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/10ee1092b0d8c92e708082487a6d2242.png'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/11-04_Waterview_blog.jpg'),
 s3.Object(bucket_name='bcbs-logo-training-images', key='gray_images/1149400efad9b15e962f01cb586f5fe3_-united-states-new-york-erie-county-orchard-park-north-buffa